In [1]:
import torch
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from models.networks.Attention_unet import AttentionUnet

In [3]:
from models.layers.grid_attention import *

In [4]:
from transforms.augmentations import *

In [ ]:
sample=np.load('/mnt/43e5e0ce-4877-4cb7-9293-17b386c78736/attention_unet/data/Only_gaussian/param_fbh_gaussian_size5mm_depth10pct_thickness3mm.npz')
x=sample['data']
y=sample['mask']

x=torch.from_numpy(x).to(torch.float64)
y=torch.from_numpy(y).to(torch.uint8)

x=x.un

AttributeError: 'Tensor' object has no attribute 'unsquezze'

In [ ]:
transforms=[
    RandomPhaseAwareSpeedChange(),
    RandomBrightnessContrast(),
    PrependFirstFrame(),
    RandomFlip3D(axes=(1,2)),
    RandomElasticTransform(),
    RandomSequenceRotation(),
    RandomCropSequence(),
    AddGaussianNoise3D()
]     